### <토픽 클러스터링 - LDA(잠재디리클래할당)>
* 텍스트 본문의 숨겨진 의미구조를 발견하기 위해 사용되는 머신러닝 비지도학습 기법 중 하나.
* 주어진 문서의 어떤 주제(topic)들이 존재하는지, 주제(topic)에 각 단어가 포함될 확률 계산
* 문서의 잠재적 의미 분석, 토픽에 의해 문서 간의 유사도 측정, 분류작업 가능
* 유전자 정보, 이미지, 네트워크 등의 자료에서 유의미한 구조를 발견하는 데에도 사용


In [ ]:
from pandas import *
import time
import sys
import re
from ckonlpy.tag import Twitter
# pip install customized_konlpy
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
#크롤링한 뉴스 기사를 명사 단위로 쪼개고, 불필요한 문장부호 제거
documents_read = read_excel('C:/Users/ahn/nounsent.xlsx')['문장단어']

documents = []
for x_gisa in documents_read:
    while "'" in x_gisa or "[" in x_gisa or "]" in x_gisa or " " in x_gisa:
        x_gisa = x_gisa.replace("'","")
        x_gisa = x_gisa.replace("[","")
        x_gisa = x_gisa.replace("]","")
        x_gisa = x_gisa.replace(" ","")
    x_words = x_gisa.split(',')
    gisa = []
    for x_word in x_words:
        gisa.append(x_word)
    documents.append(gisa)
print(documents[0])

['자카르타', '팔렘방', '아시안', '게임', '혈투', '우즈베크', '연장', '후반', '동점', '상황', '황희', '페널티킥', '지점', '이번', '대회', '찬스', '자주', '비난', '스스로', '페널티킥', '양보', '전담', '손흥민', '고개', '까지', '황희', '침착', '오른쪽', '구석', '그간', '울분', '황희', '경고', '무릅쓰', '자신', '유니폼', '사이', '골문', '주먹', '불끈', '포효', '명의', '공격수', '과감', '돌파', '페널티킥', '와일드카드', '스트라이커', '황의조', '이상', '얼굴', '경기', '요약', '황의조', '시작', '황의조', '한국', '축구', '대표팀', '아시안', '게임', '강전', '브카시', '패트리어트', '찬드라', '스타디움', '황의조', '해트트릭', '황희', '연장', '결승', '페널티킥', '우즈베키스탄', '한국', '부산', '대회', '부터', '연속', '황의조', '발끝', '뿜었다', '전반', '손흥민', '스루', '패스', '선제골', '슈팅', '골로', '연결', '전반', '공중', '한국', '수비수', '서로', '사이', '가로', '우즈베크', '동점골', '경기', '흐름', '완전', '어가', '수비', '한국', '황의조', '중거리', '기사', '회생', '전반', '황인', '범의', '패스', '황의조', '슈팅', '골망', '정도', '절정', '컨디션', '자랑', '황의조', '하나', '작품', '었다', '후반', '시작', '경기', '양상', '다시', '세로', '후반', '알리', '바예프', '노마크', '찬스', '내주', '동점', '허용', '상대', '슈팅', '현수', '골망', '중력', '한국', '수비', '자초', '결과', '승부', '추가', '그대로', '기우', '후반', '우즈베크', '수비수', '헛발질', '손흥민', '가로

In [3]:
# blog 참고
# https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/07/09/lda 블로그 참고
# 토픽 클러스터링 코드(사용자의 설정에 의해 유사 의미 단어 그룹 형성)

from collections import Counter
import random
from pandas import *

#
def p_topic_given_document(topic, d, alpha=0.1):          # A/ 문서 d의 모든 단어 중 토픽에 속하는 단어 비율(alpha 더해 smoothing)
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):            # B/ 토픽에 속한 단어 가운데 입력인자 단어의 비율
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

def topic_weight(d, word, k):                             # 문서와 문서 내 단어가 주어지면 k번째 토픽의 비중 반환 / A*B
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

def sample_from(weights):
    total = sum(weights)
    rnd = total * random.random()
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i


random.seed(0)

K=4    # 사용자가 지정하는 토픽 수, 4개의 주제를 가진 명사 집합 만들기

document_topics = [[random.randrange(K) for word in document]  # 각 단어를 임의의 토익에 랜덤 배정
                    for document in documents]
document_topic_counts = [Counter() for _ in documents]         # 각 토픽이 각 문서에 할당되는 횟수
topic_word_counts = [Counter() for _ in range(K)]              # 각 단어가 각 토픽에 할당되는 횟수
topic_counts = [0 for _ in range(K)]                           # 토픽에 할당된 총 단어수
document_lengths = [len(document) for document in documents]   # 각 문서에 포함되는 총 단어수
distinct_words = set(word for document in documents for word in document) # 단어 종류
V = len(distinct_words)
D = len(documents)  # 총 문서의 수

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

for iter in range(100):    # sample 대상 count
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):
            # 깁스 샘플링 수행을 위해 샘플링 대상 단어와 토픽 제외하고 카운트
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1
            # 깁스 샘플링 대상 단어, 토픽을 제외한 말뭉치 모든 단어의 토픽 정보를 토대로 
            # 샘플링 대상 단어의 새로운 토픽을 선택
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic
            #샘플링 대상 단어의 새로운 토픽을 반영해 말뭉치 정보 업데이트
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1
    print(iter)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
topic_word_counts

[Counter({'아시안': 51,
          '이번': 10,
          '비난': 0,
          '스스로': 1,
          '전담': 1,
          '그간': 1,
          '울분': 1,
          '황희': 21,
          '무릅쓰': 1,
          '주먹': 1,
          '돌파': 2,
          '페널티킥': 5,
          '황의조': 49,
          '경기': 43,
          '한국': 77,
          '찬드라': 0,
          '해트트릭': 8,
          '우즈베키스탄': 15,
          '손흥민': 40,
          '패스': 9,
          '전반': 14,
          '동점골': 4,
          '어가': 3,
          '수비': 10,
          '기사': 2,
          '범의': 2,
          '정도': 1,
          '자랑': 6,
          '작품': 0,
          '후반': 19,
          '알리': 4,
          '노마크': 1,
          '찬스': 6,
          '중력': 2,
          '승부': 5,
          '기우': 1,
          '골망': 5,
          '마지막': 4,
          '페널티': 1,
          '당황': 0,
          '선언': 2,
          '무실점': 2,
          '자신': 1,
          '호흡': 3,
          '동갑': 0,
          '이하': 17,
          '시절': 2,
          '사이다': 1,
          '노래': 0,
          '게임': 56,
          '마다': 3

In [27]:
documents[0]

['자카르타',
 '팔렘방',
 '아시안',
 '게임',
 '혈투',
 '우즈베크',
 '연장',
 '후반',
 '동점',
 '상황',
 '황희',
 '페널티킥',
 '지점',
 '이번',
 '대회',
 '찬스',
 '자주',
 '비난',
 '스스로',
 '페널티킥',
 '양보',
 '전담',
 '손흥민',
 '고개',
 '까지',
 '황희',
 '침착',
 '오른쪽',
 '구석',
 '그간',
 '울분',
 '황희',
 '경고',
 '무릅쓰',
 '자신',
 '유니폼',
 '사이',
 '골문',
 '주먹',
 '불끈',
 '포효',
 '명의',
 '공격수',
 '과감',
 '돌파',
 '페널티킥',
 '와일드카드',
 '스트라이커',
 '황의조',
 '이상',
 '얼굴',
 '경기',
 '요약',
 '황의조',
 '시작',
 '황의조',
 '한국',
 '축구',
 '대표팀',
 '아시안',
 '게임',
 '강전',
 '브카시',
 '패트리어트',
 '찬드라',
 '스타디움',
 '황의조',
 '해트트릭',
 '황희',
 '연장',
 '결승',
 '페널티킥',
 '우즈베키스탄',
 '한국',
 '부산',
 '대회',
 '부터',
 '연속',
 '황의조',
 '발끝',
 '뿜었다',
 '전반',
 '손흥민',
 '스루',
 '패스',
 '선제골',
 '슈팅',
 '골로',
 '연결',
 '전반',
 '공중',
 '한국',
 '수비수',
 '서로',
 '사이',
 '가로',
 '우즈베크',
 '동점골',
 '경기',
 '흐름',
 '완전',
 '어가',
 '수비',
 '한국',
 '황의조',
 '중거리',
 '기사',
 '회생',
 '전반',
 '황인',
 '범의',
 '패스',
 '황의조',
 '슈팅',
 '골망',
 '정도',
 '절정',
 '컨디션',
 '자랑',
 '황의조',
 '하나',
 '작품',
 '었다',
 '후반',
 '시작',
 '경기',
 '양상',
 '다시',
 '세로',
 '후반',
 '알리',
 '바예프',
 '노마크',
 '찬스',
 '

In [28]:
document_topic_counts[0]

Counter({3: 8, 0: 218, 2: 42, 1: 0})

In [29]:
print(document_topic_counts)

[Counter({0: 218, 2: 42, 3: 8, 1: 0}), Counter({0: 148, 2: 0, 3: 0, 1: 0}), Counter({2: 187, 0: 56, 1: 26, 3: 0}), Counter({1: 324, 0: 19, 3: 13, 2: 7}), Counter({0: 139, 1: 42, 3: 0, 2: 0}), Counter({1: 178, 0: 43, 3: 0, 2: 0}), Counter({0: 82, 2: 9, 1: 5, 3: 0}), Counter({3: 113, 1: 74, 0: 28, 2: 5}), Counter({2: 169, 0: 67, 1: 46, 3: 0}), Counter({2: 88, 1: 63, 0: 59, 3: 0}), Counter({0: 107, 1: 88, 2: 73, 3: 0}), Counter({0: 6, 2: 1, 1: 0, 3: 0}), Counter({3: 170, 2: 9, 1: 2, 0: 0}), Counter({3: 147, 2: 6, 0: 0, 1: 0}), Counter({3: 184, 2: 9, 0: 0, 1: 0}), Counter({0: 2, 3: 0, 2: 0, 1: 0}), Counter({2: 193, 1: 58, 0: 18, 3: 0}), Counter({0: 56, 3: 1, 2: 0, 1: 0}), Counter({1: 90, 0: 46, 3: 0, 2: 0}), Counter({0: 92, 1: 2, 3: 0, 2: 0}), Counter({0: 175, 1: 0, 2: 0, 3: 0}), Counter({0: 22, 2: 22, 3: 0, 1: 0}), Counter({3: 19, 0: 2, 2: 0, 1: 0}), Counter({2: 235, 3: 16, 0: 14, 1: 0}), Counter({0: 150, 2: 27, 3: 16, 1: 11}), Counter({0: 85, 3: 1, 2: 1, 1: 0}), Counter({3: 149, 1: 139, 

In [30]:
# 첫번째 토픽, 20개 데이터 내림차순정렬
sorted_d1 = sorted(topic_word_counts[0].items(), key=lambda x: x[1],reverse=True)
sorted_d1[:20]

[('한국', 69),
 ('아시안', 53),
 ('게임', 50),
 ('황의조', 49),
 ('손흥민', 48),
 ('경기', 39),
 ('대표팀', 39),
 ('축구', 35),
 ('팔렘방', 34),
 ('자카르타', 29),
 ('이란', 29),
 ('말레이시아', 29),
 ('오후', 23),
 ('선수', 23),
 ('인도네시아', 23),
 ('바레인', 22),
 ('황희', 21),
 ('남자', 20),
 ('후반', 19),
 ('강전', 18)]

In [31]:
# 두번째 토픽, 20개 데이터 내림차순정렬
sorted_d2 = sorted(topic_word_counts[1].items(), key=lambda x: x[1],reverse=True)
sorted_d2[:20]

[('선수', 35),
 ('게임', 34),
 ('축구', 28),
 ('아시안', 26),
 ('한국', 24),
 ('병역', 19),
 ('리그', 18),
 ('월드컵', 17),
 ('대회', 15),
 ('프리미어리그', 14),
 ('세계', 13),
 ('스포츠', 12),
 ('경기', 11),
 ('자카르타', 11),
 ('우리', 11),
 ('시즌', 10),
 ('이번', 9),
 ('일보', 9),
 ('야구', 9),
 ('독일', 9)]

In [32]:
# 3번째 토픽, 10개 내림차순 정렬 data 출력
sorted_d3 = sorted(topic_word_counts[2].items(), key=lambda x: x[1],reverse=True)
sorted_d3[:10]

[('축구', 45),
 ('베트남', 22),
 ('경기', 20),
 ('대표팀', 16),
 ('한국', 14),
 ('까지', 13),
 ('자신', 12),
 ('결과', 11),
 ('모습', 11),
 ('오늘', 10)]

In [33]:
# 4번째 토픽, 10개 내림차순 정렬 data 출력
sorted_d4 = sorted(topic_word_counts[3].items(), key=lambda x: x[1],reverse=True)
sorted_d4[:10]

[('후보', 32),
 ('의원', 31),
 ('이해찬', 13),
 ('손흥민', 12),
 ('대표', 11),
 ('지금', 10),
 ('송영길', 10),
 ('민주당', 10),
 ('맨유', 9),
 ('면서', 8)]